#  
# JSON structured data from tables in scientific papers

## Jsonformer non-token approach for OpenAI
#### (based on: https://github.com/1rgs/jsonformer, https://github.com/martinezpl/jsonformer/tree/add-openai)

In [1]:
import os
import pandas as pd
import json
import time

import openai
from IPython.display import IFrame
from IPython.display import HTML

from LLMsTablesToJson.html_table import get_driver, extract_tableSource, extract_table, quit_driver
from LLMsTablesToJson.jsonformer_non_tokens import JsonformerNoTokens, OpenAIModel, highlight_values

DATA_DIR = "your_path"
chromedriver_path = os.path.join(DATA_DIR, "chromedriver.exe")

openai.api_key = "your_openai_api_key"

## Example: MOFs properties

### Get table HTML code from DOI

In [76]:
doi = "10.1021/ja4045289"

In [77]:
IFrame(src="https://dx.doi.org/" + doi, width=900, height=380)

In [56]:
driver = get_driver(doi, chromedriver_path)
time.sleep(2)
tableSource = extract_tableSource(driver)

[<selenium.webdriver.remote.webelement.WebElement (session="5564dd04fb9d40bac2463494196628c5", element="AB458CADFFAE117495615AF8F248A6E8_element_133")>, <selenium.webdriver.remote.webelement.WebElement (session="5564dd04fb9d40bac2463494196628c5", element="AB458CADFFAE117495615AF8F248A6E8_element_134")>]
number of tables = 2


In [57]:
table_num = 1
html_table_MOFproperties = extract_table(tableSource, table_num)
quit_driver(driver)

In [58]:
html_table_MOFproperties

'<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MOFs</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">N<sub>2</sub></th><th class="colsep0 rowsep0" scope="col" align="center" char=".">calcd</th><th class="c

In [59]:
with open("html_table_MOFproperties.txt", "w") as file:
    file.write(html_table_MOFproperties)

### Show table

In [61]:
HTML(html_table_MOFproperties)

### Convert HTML table to JSON format with openAI LLMs

In [62]:
json_schema_MOFproperties = {
    "type": "object",
    "properties": {
        "MOFs": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "MOF": {"type": "string"},
                    "N2_BET": {"type": "number"},
                    "calcd_BET": {"type": "number"},
                    "N2_Vpore": {"type": "number"},
                    "CH4_Vpore": {"type": "number"},
                    "calcd_Vpore": {"type": "number"},
                    "ρ": {"type": "number"},
                    "metal": {"type": "number"},
                },
            },
        },
    },
}

In [63]:
with open('json_schema_MOFproperties.json', 'w') as file:
    json.dump(json_schema_MOFproperties, file)

In [72]:
with open("json_schema_MOFproperties.json") as file:
    json_schema_MOFproperties = json.load(file)
    file.close()

In [64]:
json_schema_MOFproperties

{'type': 'object',
 'properties': {'MOFs': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'MOF': {'type': 'string'},
     'N2_BET': {'type': 'number'},
     'calcd_BET': {'type': 'number'},
     'N2_Vpore': {'type': 'number'},
     'CH4_Vpore': {'type': 'number'},
     'calcd_Vpore': {'type': 'number'},
     'ρ': {'type': 'number'},
     'metal': {'type': 'number'}}}}}}

In [69]:
builder = JsonformerNoTokens(
    model=OpenAIModel("text-davinci-003", debug=False),
    json_schema=json_schema_MOFproperties,
    text=html_table_MOFproperties,
    prompt="Generate a object with the following schema extracting the information from the provided table in html code:",
    temperature=0.1,
    debug=True,
    max_array_length=6,
    max_string_token_length=10,
)

print("Generating...")
result_MOFproperties = builder()

Generating...
[generate_string] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MOFs</th><th class="colsep0 ro

[generate_number] response  1850, "calcd_
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MO

[generate_number] response  0.78, "CH
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MOFs</

[generate_number] response  0.78, "\u
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MOFs</

[generate_number] response  4.38}, {"MO
[generate_string] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MOFs

...

[generate_number] response  1.82}, {"MO
[generate_string] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MOFs

[generate_number] response  4930, "calcd
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MOF

[generate_number] response  2.09, "CH
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MOFs</

[generate_number] response  2.03, "\u
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table class="table" border="0"><colgroup><col align="left"><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."><col align="char" char="."></colgroup><thead><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">&nbsp;</th><th class="rowsep1 colsep0" colspan="2" scope="col" align="center" char=".">BET (m<sup>2</sup>/g)</th><th class="rowsep1 colsep0" colspan="3" scope="col" align="center" char="."><i>V</i><sub>pore</sub> (cc/g)</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th><th class="colsep0 rowsep0" scope="col" align="center" char=".">&nbsp;</th></tr><tr valign="top" class="colsep0"><th class="colsep0 rowsep0" scope="col" align="center">MOFs</

[generate_number] response  1.36}]}


In [70]:
highlight_values(result_MOFproperties)

{
  MOFs: [
    {
      MOF: "HKUST-1",
      N2_BET: 1850,
      calcd_BET: 2064,
      N2_Vpore: 0.78,
      CH4_Vpore: 0.78,
      calcd_Vpore: 0.78,
      ρ: 0.883,
      metal: 4.38
    },
    {
      MOF: "Ni-MOF-74",
      N2_BET: 1350,
      calcd_BET: 1240,
      N2_Vpore: 0.51,
      CH4_Vpore: 0.52,
      calcd_Vpore: 0.49,
      ρ: 1.206,
      metal: 7.74
    },
    {
      MOF: "PCN-14",
      N2_BET: 2000,
      calcd_BET: 2170,
      N2_Vpore: 0.85,
      CH4_Vpore: 0.78,
      calcd_Vpore: 0.76,
      ρ: 0.829,
      metal: 2.59
    },
    {
      MOF: "UTSA-20",
      N2_BET: 1620,
      calcd_BET: 1960,
      N2_Vpore: 0.66,
      CH4_Vpore: 0.66,
      calcd_Vpore: 0.69,
      ρ: 0.909,
      metal: 3.61
    },
    {
      MOF: "NU-125",
      N2_BET: 3120,
      calcd_BET: 3680,
      N2_Vpore: 1.29,
      CH4_Vpore: 1.23,
      calcd_Vpore: 1.32,
      ρ: 0.578,
      metal: 1.82
    },
    {
      MOF: "NU-111",
      N2_BET: 4930,
      calcd_BET: 4650,
      N2

In [ ]:
with open('json_table_MOFproperties.json', 'w') as file:
    json.dump(result_MOFproperties, file)